Web server interface at https://xxxx:7473 Update - since the videos were filmed, neo4j requires a longer, more complex password, so the newest password is here: Username: neo4j

Password: ucb_mids_w205


In [1]:
import neo4j

import pandas as pd

from IPython.display import display

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")

# Minimum Spanning Tree

In [34]:
my_neo4j_wipe_out_database()

query = """

LOAD CSV WITH HEADERS FROM 'file:///cor.csv' AS row
MERGE (a:Stock {name: row.stock_a})
MERGE (b:Stock {name: row.stock_b})
MERGE (a)-[:LINK {weight: toFloat(row["0"])}]->(b)
MERGE (b)-[s:LINK {weight: toFloat(row["0"])}]->(a)
"""

session.run(query)

In [8]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,A,[Stock]
1,AAL,[Stock]
2,AAP,[Stock]
3,AAPL,[Stock]
4,ABBV,[Stock]
...,...,...
425,XYL,[Stock]
426,YUM,[Stock]
427,ZBH,[Stock]
428,ZION,[Stock]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,A,[Stock],LINK,AAL,[Stock]
1,A,[Stock],LINK,AAP,[Stock]
2,A,[Stock],LINK,AAPL,[Stock]
3,A,[Stock],LINK,ABBV,[Stock]
4,A,[Stock],LINK,ABT,[Stock]
...,...,...,...,...,...
184465,ZTS,[Stock],LINK,XRX,[Stock]
184466,ZTS,[Stock],LINK,XYL,[Stock]
184467,ZTS,[Stock],LINK,YUM,[Stock]
184468,ZTS,[Stock],LINK,ZBH,[Stock]


-------------------------
  Density: 2.0
-------------------------


In [9]:
query = "CALL gds.graph.drop('graph', false)"
session.run(query)

query = """

CALL gds.graph.project(
  'graph', 
  'Stock',   
  {
    LINK: {
        properties: 'weight',
        orientation: 'UNDIRECTED'
    }
  }
)

"""

session.run(query)

In [10]:
query = """

MATCH (n:Stock {name: $source})
CALL gds.spanningTree.write('graph', {
  sourceNode: n,
  relationshipWeightProperty: 'weight',
  writeProperty: 'writeCost',
  writeRelationshipType: 'MINST'
})
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;

"""

source = "AAPL"

my_neo4j_run_query_pandas(query, source=source)

,preProcessingMillis,computeMillis,writeMillis,effectiveNodeCount
0,1,81,81,430


In [11]:

query = """

MATCH path = (n:Stock {name: $source})-[:MINST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS source, endNode(rel).name AS destination, rel.writeCost AS cost

"""

source = "AAPL"

my_neo4j_run_query_pandas(query, source=source)

,source,destination,cost
0,AAPL,CHK,-0.777350
1,CHK,BA,-0.721650
2,CHK,RHI,-0.698809
3,CHK,GT,-0.625741
4,CHK,AKAM,-0.796752
...,...,...,...
424,XOM,ADBE,-0.779078
425,ADBE,CF,-0.719713
426,ADBE,COP,-0.610530
427,ADBE,MO,-0.447406


#### Find the central nodes

In [12]:
query = "CALL gds.graph.drop('graph2', false)"
session.run(query)

query = "CALL gds.graph.project('graph2', 'Stock', 'MINST', {relationshipProperties: 'writeCost'})"
session.run(query)

In [13]:
query = """

CALL gds.degree.stream('graph2', { orientation: 'NATURAL' })
YIELD nodeId, score
WHERE score > 2
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree ASC, name

"""

df = my_neo4j_run_query_pandas(query)
df

,name,degree
0,AAPL,3.0
1,AEE,3.0
2,AON,3.0
3,BIIB,3.0
4,PG,3.0
5,PX,3.0
6,RRC,3.0
7,XEL,3.0
8,ADBE,4.0
9,LEG,4.0


In [48]:
print(list(df.name))

['AAPL', 'AEE', 'AON', 'BIIB', 'PG', 'PX', 'RRC', 'XEL', 'ADBE', 'LEG', 'UNH', 'WMT', 'CSCO', 'HP', 'SBUX', 'UNM', 'BDX', 'PDCO', 'HAL', 'CHK', 'EQT', 'EXC', 'APA', 'CHTR', 'PFE', 'HUM']


# LOUVAIN MODULARITY

#### For every node, delete all but the three highest correlation outgoing relationships

In [26]:
query = """

MATCH (n:Stock)
WITH n
MATCH (n)-[r:LINK]->()
WITH n, r, COUNT(*) AS totalLinks
ORDER BY r.weight DESC
WITH n, totalLinks, COLLECT(r) AS relationships
WITH n, totalLinks, relationships[3..] AS relsToDelete
UNWIND relsToDelete AS relToDelete
DELETE relToDelete;


"""

session.run(query)

In [27]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,A,[Stock]
1,AAL,[Stock]
2,AAP,[Stock]
3,AAPL,[Stock]
4,ABBV,[Stock]
...,...,...
425,XYL,[Stock]
426,YUM,[Stock]
427,ZBH,[Stock]
428,ZION,[Stock]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,A,[Stock],LINK,SHW,[Stock]
1,A,[Stock],LINK,SRCL,[Stock]
2,A,[Stock],LINK,WAT,[Stock]
3,AAL,[Stock],LINK,EFX,[Stock]
4,AAL,[Stock],LINK,INTU,[Stock]
...,...,...,...,...,...
1285,ZION,[Stock],LINK,CMA,[Stock]
1286,ZION,[Stock],LINK,EXR,[Stock]
1287,ZTS,[Stock],LINK,EXPD,[Stock]
1288,ZTS,[Stock],LINK,EXR,[Stock]


-------------------------
  Density: 0.0
-------------------------


In [28]:
query = "CALL gds.graph.drop('graph', false)"
session.run(query)
query = """

CALL gds.graph.project(
  'graph',
  'Stock',
  'LINK'
) 
YIELD *

"""
session.run(query)


In [29]:
query = """
CALL gds.louvain.write('graph', {writeProperty: 'louvain'}) YIELD *

"""
session.run(query)

#### Color the nodes according to their community

In [30]:
query = """
MATCH (n:Stock)
WITH DISTINCT toString(n.louvain) AS stockGroup, collect(DISTINCT n) AS stocks
CALL apoc.create.addLabels(stocks, [apoc.text.upperCamelCase(stockGroup)]) YIELD node
RETURN *

"""

session.run(query)

In [31]:
query = """
MATCH (n:Stock)
RETURN n.name as Stock, n.louvain as Community
ORDER BY Community, Stock

"""

df = my_neo4j_run_query_pandas(query)
df

,Stock,Community
0,AAP,30
1,AIG,30
2,AZO,30
3,CAT,30
4,CI,30
...,...,...
425,VFC,310
426,XRAY,310
427,NAVI,331
428,PRGO,331


In [32]:
df.Community.value_counts()

36     89
249    71
251    69
30     55
280    53
310    33
183    30
44     27
331     3
Name: Community, dtype: int64

In [33]:
query = """

MATCH (n:Stock)
RETURN n.louvain AS communityId,
       count(*) AS communitySize,
       collect(n.name) AS Names
ORDER BY communitySize DESC

"""

my_neo4j_run_query_pandas(query)

,communityId,communitySize,Names
0,36,89,"[MMM, AES, AFL, ARE, GOOGL, GOOG, AMT, ADI, AI..."
1,249,71,"[AOS, A, ALK, AMP, AME, APH, AVY, BAC, COF, CN..."
2,251,69,"[ABBV, AYI, AMD, ALL, AMZN, AXP, AON, AMAT, AJ..."
3,30,55,"[AAP, AIG, AZO, KMX, CAT, CI, CINF, STZ, CMI, ..."
4,280,53,"[ABT, ACN, AMG, ALGN, ALLE, AAL, AMGN, ANSS, A..."
5,310,33,"[ADBE, AKAM, LNT, AEP, AWK, AAPL, AVB, CNP, CM..."
6,183,30,"[ALB, MO, APA, APTV, BDX, BWA, BMY, CF, CHK, C..."
7,44,27,"[APD, AEE, ADM, BIIB, CHRW, CHTR, CMCSA, EQT, ..."
8,331,3,"[NAVI, PRGO, WBA]"


# Low Degree Centrality Portfolio

### PageRank

In [57]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Stock', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [27]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank ASC, name ASC

"""

max_iterations = 20
damping_factor = 0.05

my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)

,name,page_rank
0,AEP,0.950000
1,AFL,0.950000
2,AIV,0.950000
3,ALL,0.950000
4,AMD,0.950000
...,...,...
425,PNC,1.189511
426,AMP,1.192319
427,APH,1.198010
428,EFX,1.202183


### Incoming Degree Centrality

In [16]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Stock', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [17]:
query = """

CALL gds.degree.stream('ds_graph', { orientation: 'REVERSE' })
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree ASC, name

"""
df = my_neo4j_run_query_pandas(query)
df

,name,degree
0,AEP,0.0
1,AFL,0.0
2,AIV,0.0
3,ALL,0.0
4,AMD,0.0
...,...,...
425,AMP,14.0
426,APH,14.0
427,PNC,14.0
428,EFX,15.0


##### Stocks that don't figure in the top three most correlated of other stocks

In [18]:
df[df.degree==0]

,name,degree
0,AEP,0.0
1,AFL,0.0
2,AIV,0.0
3,ALL,0.0
4,AMD,0.0
...,...,...
66,VFC,0.0
67,VRSK,0.0
68,WY,0.0
69,WYNN,0.0
